In [1]:
import sys
import os

# Add 'src' folder to the system path
sys.path.append(os.path.abspath("src"))

# Import necessary libraries
import requests
import pandas as pd
from tqdm import tqdm
import time
import utils

from dotenv import load_dotenv
load_dotenv()


True

In [2]:
# GitHub repository details
OWNER = "wso2"  
REPO = "product-apim"  

# GitHub API token (Required for Events API to avoid rate limiting)
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
if not GITHUB_TOKEN:
    raise ValueError("GITHUB_TOKEN environment variable is not set.")

HEADERS = {
    "Authorization": f"Bearer {GITHUB_TOKEN}"
}

In [4]:
def run_query(query, variables):
    response = requests.post(
        'https://api.github.com/graphql',
        json={'query': query, 'variables': variables},
        headers=HEADERS
    )
    if response.status_code != 200:
        raise Exception(f"Query failed: {response.text}")
    return response.json()

GRAPHQL_QUERY = """
query ($queryString: String!, $cursor: String) {
  search(query: $queryString, type: ISSUE, first: 100, after: $cursor) {
    pageInfo {
      endCursor
      hasNextPage
    }
    edges {
      node {
        ... on PullRequest {
          number
          title
          body
          url
          linkedIssues(first: 10) {
            nodes {
              number
              title
            }
          }
        }
      }
    }
  }
}
"""

def fetch_all_linked_prs(repo):
    query_string = f'is:pr linked:issue repo:{repo}'
    cursor = None
    all_prs = []

    while True:
        variables = {
            "queryString": query_string,
            "cursor": cursor
        }
        result = run_query(GRAPHQL_QUERY, variables)
        print(result);
        search_data = result['data']['search']

        for edge in search_data['edges']:
            node = edge['node']
            if node:
                all_prs.append(node)

        if not search_data['pageInfo']['hasNextPage']:
            break

        cursor = search_data['pageInfo']['endCursor']
        time.sleep(0.5)  # Be polite to GitHub's API

    return all_prs


all_linked_prs = fetch_all_linked_prs(REPO)
print(f"Fetched {len(all_linked_prs)} linked PRs.")

# Example: print PR numbers
for pr in all_linked_prs:
    print(f"PR #{pr['number']} - {pr['title']}")


{'errors': [{'path': ['query', 'search', 'edges', 'node', '... on PullRequest', 'linkedIssues'], 'extensions': {'code': 'undefinedField', 'typeName': 'PullRequest', 'fieldName': 'linkedIssues'}, 'locations': [{'line': 15, 'column': 11}], 'message': "Field 'linkedIssues' doesn't exist on type 'PullRequest'"}]}


KeyError: 'data'